# RES Cleaning

This notebook walks through how the RES articles were sorted into categories of articles and non-articles.

## Load Libraries

In [25]:
from tokenize import Ignore
import pandas as pd
import time
from os import path
import sys
from pathlib import Path
from PyPDF2 import PdfFileReader, PdfFileWriter
import re
import os
from difflib import SequenceMatcher
import datetime
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows',100)
pd.set_option('display.max_colwidth', 200)    

## Load Files

Replace the file paths below to match local file paths. Note: comment out files that are not available eg: datadumps.

In [26]:
base_path="/Users/sijiawu/Work/Thesis/Data"

In [27]:
masters = pd.read_excel(base_path+"/Masterlists/RESTUD_Masterlist.xlsx")
masters10 = pd.read_excel(base_path+ "/2010/RESTUD_master.xlsx")
pivots = pd.read_excel(base_path+"/Pivots/RESTUD_Pivots2020.xlsx")

## Create File Names

In [28]:
saveas=base_path+"/Processed/RES_processed.xlsx"

## Some random checks on the masters list

My assumption is that all data without author names must be miscellaneous documents like reports by the committee, forewords, front matters etc.. The goal of this notebook is to check for certain that all the documents without author names are actually miscellaneous documents and then classify them as miscellaneous (MISC). Hence, first we group everything the data by title to see the repetitive general content that can likely be removed.

In [29]:
pd.set_option('display.max_rows',masters.shape[0])
temp=masters['title'].str.lower().value_counts()
pd.DataFrame(temp[temp>1])

,count
title,
front matter,303
back matter,284
volume information,73
accepted manuscripts,9
books and monographs received,8
periodicals received,8
editorial announcement,8
editorial,5
comment,5


Some repetitions are due to multiple comments. Now consider this list in absence of author names.

In [30]:
temp1=masters[masters['author'].isna()]['title'].str.lower().value_counts()
pd.DataFrame(temp1)

,count
title,
front matter,303
back matter,284
volume information,73
accepted manuscripts,9
periodicals received,8
books and monographs received,8
announcement,3
books and periodicals received,3
introduction,2


In [31]:
#pd.DataFrame(masters[masters['content_type'].isna()]['title'].str.lower().value_counts())
#masters[masters['title'].str.lower().str.match(r'(^|: )report of the')]
#masters[masters['title'].str.lower().str.match(r'(^|.*: )report of the')]
#masters.loc[masters['title'].str.lower().str.match(r'^combined references(.*)')==True,'content_type']='MISC'

# this is a random panel for testing code
masters[masters["title"].isna()==True]

,type,issue_url,ISSN,URL,journal,number,publisher,title,urldate,volume,year,abstract,author,pages,reviewed-author,uploaded


Notice that titles with duplicates of greater or equal to 4 are miscellaneous items and so they are classified in bulk

In [32]:
temp2=masters['title'].str.lower().value_counts()
removal=list(temp2[temp2>=8].index)
masters.loc[masters.title.str.lower().isin(removal),'content_type']='MISC'

## Combine the scraped list with the citations files
This is to rename the fields to differentiate scopus and jstor data fields and drop some miscellaneous columns that are not necessary. There are no reviews in RES (ironically) or atleast what jstor classifies as reviews.

In general, I have found that masterlists contructed from citation files lack the reviewed source's name while it is present on the page of the article. Some files are also just missing the title. So I'm combining the old masterlists with the new ones.

In [33]:
#preprocessing step
pivots['type']=pd.NA
masters["URL"]="https:"+masters["URL"].str.split(':').str[-1]
masters.drop('type', inplace=True, axis=1)
masters10["stable_url"]="https:"+masters10["stable_url"].str.split(':').str[-1]
masters10.rename(columns = {'authors':'author','stable_url':'URL','title':'title_10'}, inplace = True)
masters['pages']=masters['pages'].str.strip()  
masters.loc[masters.title.str.lower() == "back matter", 'pages'] = pd.NA  
masters['pages']=masters["pages"].str.split('pp. ').str[-1]
masters['pages']=masters['pages'].replace(r'--','-',regex=True).str.strip()

In [34]:
masters["author_split"]=masters['author'].str.split(' and ')


In [35]:
masters=masters.merge(masters10[['URL', 'title_10']], how='left', on='URL')
masters.loc[(pd.isna(masters["title"])==True)&(pd.isna(masters["reviewed-author"])==False), 'content_type']="Review"

Classify some titles via regex

In [36]:
masters.loc[masters['title'].str.lower().str.contains('books and periodicals')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^editorial')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^announcement')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^author index')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^index to volumes.*')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^index$')==True,'content_type']='MISC'
masters.loc[(masters['title'].str.lower().str.match(r'^introduction$')==True)&(masters['author'].isna()==True),'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^subject index$')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.contains(r'econometric society')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.contains(r'periodicals')==True,'content_type']='MISC'

## Classifying other content

In [37]:
sum(masters.content_type.isna())
#masters.shape[0]

3422

In [38]:
masters.loc[masters['author'].str.lower().str.match(r'^review(ed|) by(.*)')==True,'content_type']='Review' #reviews
masters.loc[(masters['title'].str.lower().str.match(r'(.*) by (.*)')==True) & (masters.author.isna()==True),'content_type']='Review2' 
#possible reviews that don't have author names
masters[(masters['content_type']=='Review2') | (masters['content_type']=='Review')].shape[0] #reviews

0

In [39]:
masters.loc[masters['title'].str.lower().str.contains("erratum")|masters['title'].str.lower().str.contains("errata"), 'content_type']="Errata"

In [40]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?).*comment.*$')==True,'content_type']='Comment'
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*comment$')==True,'content_type']='Comment'
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'(a further|further) comment.*$')==True,'content_type']='Comment'
masters[masters['content_type']=='Comment'].shape[0] 
#comments

60

In [41]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?) (|a )reply(| to.*)$')==True,'content_type']="Reply"
masters[masters['content_type']=='Reply'].shape[0]

15

In [42]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?|).*rejoinder.*$')==True,'content_type']="Rejoinder"
masters[masters['content_type']=='Rejoinder'].shape[0]


9

In [43]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*: (|a )discussion$')==True,'content_type']="Discussion"
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'(^|a )discussion(|.*)$')==True,'content_type']="Discussion"
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*:.*(|a )discussion(|s)$')==True,'content_type']='Discussion'
masters[masters['content_type']=='Discussion'].shape[0]


0

In [44]:
masters.loc[masters['content_type'].isna(),'content_type']="Article"
masters[masters['content_type']=='Article'].shape[0]

3312

In [45]:
# block for testing regex patterns
#masters[masters['title'].str.lower().str.match(r'^\washington notes$')==True]
#masters[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?) (|a )rejoinder.*$')==True]
#masters[masters.content_type=='Discussion'].shape[0]

## Consider the pivots file
At times, conference papers are structured differently to normal articles. Hence, it may be necessary to distinguish conference papers from common articles. The next code block separates special issues (S) from normal issues (N)

In [46]:
pivots.loc[pivots.Jstor_issue_text.isna(),"Jstor_issue_text"]="None"
pivots.loc[pivots.Jstor_issue_text.str.lower().str.match(r'(.*)(special issue|symposium|index to volumes|survey)(.*)'),'type']="S"
pivots.loc[pivots.type.isna(),'type']='N'
pivots.type.value_counts()

type
N    313
S      8
Name: count, dtype: int64

Merge masterlist and pivot list

In [47]:
result = pd.merge(masters, pivots[['issue_url','type']], how="left", on=["issue_url", "issue_url"])

In [48]:
result.to_excel(saveas, index=False)